In [3]:
!pip install -q -U accelerate
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U langchain

In [15]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=3a7efd831e7c1c15d5930633d3215a298fac00207167db0635759b553c5b2579
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [18]:
!pip install qdrant-client

  Obtaining dependency information for qdrant-client from https://files.pythonhosted.org/packages/c6/2e/abb6befe93d321904d05d6667762f9a9fb59af1b144de535f50e1ec0cc14/qdrant_client-1.6.9-py3-none-any.whl.metadata
  Obtaining dependency information for grpcio-tools>=1.41.0 from https://files.pythonhosted.org/packages/d3/df/a51af6461494efc9ecf5379967aed75006b8b58a2fb3dfc6a0f48a7d9d11/grpcio_tools-1.59.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for httpx[http2]>=0.14.0 from https://files.pythonhosted.org/packages/a2/65/6940eeb21dcb2953778a6895281c179efd9100463ff08cb6232bb6480da7/httpx-0.25.2-py3-none-any.whl.metadata
  Obtaining dependency information for portalocker<3.0.0,>=2.7.0 from https://files.pythonhosted.org/packages/17/9e/87671efcca80ba6203811540ed1f9c0462c1609d2281d7b7f53cef05da3d/portalocker-2.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for protobuf<5.0dev,>=4.21.6 from https://files.pythonhosted.o

In [4]:
import pandas as pd
from langchain.vectorstores import Qdrant as Qdrant1
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
import torch
from transformers import BitsAndBytesConfig

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
data = pd.read_csv('/kaggle/input/bigbasket/bigBasketProducts.csv')

In [6]:
data.head()

,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description
0,1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
1,2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
2,3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
3,4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
4,5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


In [7]:
print(data.dtypes)
print(len(data))

index             int64
product          object
category         object
sub_category     object
brand            object
sale_price      float64
market_price    float64
type             object
rating          float64
description      object
dtype: object
27555


In [8]:
print(data["product"].unique().shape)

(23541,)


In [9]:
# Combine columns into a meaningful descriptive document
def create_descriptive_document(row):
    # Example: Concatenate 'product_name', 'brand', 'category', 'description' columns
    document = f"Product name is: {row['product']}\n"
    document += f"Brand: {row['brand']}\n"
    document += f"Category: {row['category']}\n"
    document += f"Description: {row['description']}\n"
    document += f"sale_price: {row['sale_price']}\n"
    document += f"has Rating: {row['rating']}\n"
    # Add more columns or modify the concatenation to suit your requirements
    return document

# Apply the function to create descriptive documents for each row
data['combined_description'] = data.apply(create_descriptive_document, axis=1)

In [10]:
data['combined_description'][0]

'Product name is: Garlic Oil - Vegetarian Capsule 500 mg\nBrand: Sri Sri Ayurveda \nCategory: Beauty & Hygiene\nDescription: This Product contains Garlic Oil that is known to help proper digestion, maintain proper cholesterol levels, support cardiovascular and also build immunity.  For Beauty tips, tricks & more visit https://bigbasket.blog/\nsale_price: 220.0\nhas Rating: 4.1\n'

In [11]:
# embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
doc_list=data['combined_description'].tolist()
# Generate embeddings for the combined descriptions
sample_list=doc_list[:500]
# embeddings = embedding_model.encode(sample_list)

In [12]:
# loader = TextLoader("/content/test.txt")
# documents = loader.load()
# print(documents)
dl1=[]
for i in range(0,len(doc_list)):
  # print(len(embeddings[i].tolist()))
  dl1.append(Document(page_content=doc_list[i], metadata={"source": "local"}))
# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# docs = text_splitter.split_documents(documents)

In [13]:
print(len(dl1))

27555


In [16]:
embeddings1 = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [19]:
qdrant1 = Qdrant1.from_documents(
    dl1,
    embeddings1,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="my_documents",
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

# LLM

In [20]:

# from llama_index.prompts import PromptTemplate
# from llama_index.llms import HuggingFaceLLM

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model_name = "tiiuae/falcon-7b-instruct"
# If GPU available use it
device = "cuda" if torch.cuda.is_available() else "cpu"
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map='auto',
    quantization_config= quantization_config
)
# Set to eval mode


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

...

pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [22]:
model.eval()
pipe = transformers.pipeline(
    "text-generation",
    model=model,
    max_new_tokens=100,
    temperature=0.1,
    tokenizer=tokenizer,
    repetition_penalty=1.5,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    eos_token_id=tokenizer.eos_token_id,

)

In [27]:

hf_pipeline = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import transformers
# import torch

# model = "tiiuae/falcon-7b-instruct"

# tokenizer = AutoTokenizer.from_pretrained(model)
# model.eval()
# pipeline =transformers.pipeline(task="text-generation", model=model, tokenizer=tokenizer,
#      max_new_tokens=100, torch_dtype=torch.bfloat16,trust_remote_code=True,device_map="auto",
#                          repetition_penalty=1.1, model_kwargs={
#                           "max_length": 1200, "temperature": 0.01,}
# )

In [28]:
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.prompts import PromptTemplate
# template = """Answer the question as truthfully as possible using the provided text, and if the answer is not contained within the text below, say "I don't know"

# Context:
# {context}

# {query}""".strip()
template = """You are assitent chatbot who gives answer only from context provided to you. Answer the question  and if the answer is not contained within the text below, say "I dont have information about it in my current Knowledge"
{summaries}
Query:{question}
"""

qa = RetrievalQAWithSourcesChain.from_chain_type(llm=hf_pipeline, chain_type="stuff",
                                 retriever=qdrant1.as_retriever(search_kwargs={"k": 3}),
                                  return_source_documents=True,
                                  chain_type_kwargs={
                                "prompt": PromptTemplate(
                                template=template,
                                input_variables=["summaries","question"],
                                ),
                                },
                                 verbose=False,)

In [29]:
import time
def llmquery(query):
    start_time = time.time()
    try:
        # Record the start time
        
        response = qa({"question": query})
    except ValueError as e:
        response = str(e)
        print(response)
        if not response.startswith("Could not parse LLM output: `"):
            raise e
        response = response.removeprefix("Could not parse LLM output: `").removesuffix("`")
    elapsed_time = time.time() - start_time
    print(elapsed_time)
    return response

In [30]:
query = "tell me the brands name whose soup rate is less tha 100?"
# Send question as a query to qa chain
result = llmquery(query)
print(result['answer'])
print("===================")
# print(result["source_documents"][0].page_content)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


3.839811325073242
The brands name whose soup rate is less than 100 is Knorr.


In [32]:
query = "Describe Multani Mati of brand Satinance?"
# Send question as a query to qa chain
result = llmquery(query)
print(result["answer"])
print("===================")
# print(result["source_documents"][0].page_content)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


4.820709705352783
Answer: Multani Mati is a skin toner and astringent that is used to reduce oiliness and keep the skin soft and clean. It also has a range of other benefits, including improving skin complexion and nourishment.


In [33]:
query = "tell me some product name masala and spices product?"
# Send question as a query to qa chain
result = llmquery(query)
print(result["answer"])
print("===================")
# print(result["source_documents"])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


8.938727378845215
There are many products that fall under the category of masala and spices. Some popular ones include:

- Garam Masala
- Chilli Powder
- Cumin Powder
- Coriander Powder
- Paprika Powder
- Cinnamon Powder
- Cardamom Powder
- Cloves Powder
- Nutmeg Powder
- Mace Powder
- Saffron Powder
- Cumin Powder
- Coriander Powder
- Paprika Powder
- Cinnamon Powder
- Cardamom Powder



In [34]:
query = "most trusted tublight brand?"
# Send question as a query to qa chain
result = llmquery(query)
print(result["answer"])
print("===================")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


5.393445014953613

The most trusted brand for LED bulbs is Philips. They have been in the industry for over 100 years and have a reputation for producing high-quality products. Their LED bulbs are energy-efficient and long-lasting, making them a popular choice among consumers.


In [35]:
query = "suggest some Hair Oil & Serum?"
# Send question as a query to qa chain
result = llmquery(query)
print(result["answer"])
print("===================")
# print(result["source_documents"])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


11.36078143119812
There are many hair oils and serums available in the market. Some of the popular ones are:

1. Moroccanoil Hair Treatment
2. L'Oreal Paris Elvive Extraordinary Oil
3. OGX Renewing + Argan Oil Of Morocco
4. Shea Moisture Coconut & Hibiscus Curl Enhancing Smoothie
5. Aveeno Absolutely Ageless Hair Care
6. Neutrogena Hydro Boost Hair Serum
7. Redken Extreme Length



#API wrap

In [1]:
!pip install flask jsonify flask_ngrok

  Preparing metadata (setup.py) ... done
  Created wheel for jsonify: filename=jsonify-0.5-py3-none-any.whl size=1538 sha256=206bb7f301e0b7cc630a971faf395d42dc99fc7ad2391b43d014164b6e1120f5
  Stored in directory: /root/.cache/pip/wheels/4e/6d/b2/9cb590fb5654ff13457f49d4aa5b16469419681613c46a2ec1
Successfully built jsonify


In [2]:

from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request

In [4]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route('/generate', methods=['POST'])
def generate_text():
    data = request.get_json()
    prompt_text = data['prompt'] if 'prompt' in data else None

    if prompt_text:
        generated_sequence = llmquery(prompt_text)
        return jsonify({'generated_text': generated_sequence['result']})
    else:
        return jsonify({'error': 'Prompt text is missing.'}), 400
print("server started")


app.run()

server started
 * Serving Flask app '__main__'
 * Debug mode: off
 * Running on http://32cf-34-90-66-166.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


In [ ]:
!python /kaggle/input/run-py/main.py